# 預處理

In [ ]:
from keras.utils import np_utils
import numpy as np
np.random.seed(123)

## Import data

In [ ]:
x_train_image = 
y_train_label = 
x_test_image = 
y_test_label = 

In [ ]:
print("train data:",'images:',X_img_train.shape," labels:",y_label_train.shape) 
print("test data:",'images:',X_img_test.shape ," labels:",y_label_test.shape) 

正規化

In [ ]:
X_img_train_normalize = X_img_train.astype('float32') / 255.0
X_img_test_normalize = X_img_test.astype('float32') / 255.0

onehotecoding

In [ ]:
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)

In [ ]:
y_label_test_OneHot.shape

# 建模

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [ ]:
model = Sequential()

## 第一層（卷積與池化）

In [ ]:
model.add(Conv2D(filters=32,kernel_size=(3, 3),input_shape=(32, 32,3),
                 activation='relu', padding='same'))
model.add(Dropout(0.3))
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                 activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

## 第二層（卷積與池化）

In [ ]:
model.add(Conv2D(filters=64, kernel_size=(3, 3),
                 activation='relu', padding='same'))
model.add(Dropout(0.3))
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

## 建立神經網路(平坦層、隱藏層、輸出層)
中間層必選reLu  
最後一層 只有一個選擇sigmoid 多個選softmax


In [ ]:
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(2500, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

## 看Summary

In [ ]:
print(model.summary())

# 訓練模型

In [ ]:
try:
    model.load_weights("SaveModel/bodyCnnModelnew.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [ ]:
train_history=model.fit(X_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.2,
                        epochs=1, batch_size=2, verbose=1)  

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_acc,test_acc):
    plt.plot(train_history.history[train_acc])
    plt.plot(train_history.history[test_acc])
    plt.title('Train History')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
show_train_history('acc','val_acc')

In [ ]:
show_train_history('loss','val_loss')

# 評估模型

In [ ]:
scores = model.evaluate(X_img_test_normalize, 
                        y_label_test_OneHot,verbose=0)
scores[1]

# 進行預測

In [ ]:
prediction=model.predict_classes(X_img_test_normalize)

In [ ]:
prediction[:2]

In [ ]:
label_dict={0:"thin",1:"medium",2:"large"}

In [ ]:
import matplotlib.pyplot as plt
def plot_images_labels_prediction(images,labels,prediction,
                                  idx,num=10):
    fig = plt.gcf()
    fig.set_size_inches(12, 14)
    if num>25: num=25 
    for i in range(0, num):
        ax=plt.subplot(5,5, 1+i)
        ax.imshow(images[idx],cmap='binary')
                
        title=str(i)+','+label_dict[labels[i][0]]
        if len(prediction)>0:
            title+='=>'+label_dict[prediction[i]]
            
        ax.set_title(title,fontsize=10) 
        ax.set_xticks([]);ax.set_yticks([])        
        idx+=1 
    plt.show()

In [ ]:
plot_images_labels_prediction(X_img_test_normalize,y_label_test,
                              prediction,0,10)

# 查看預測機率

In [ ]:
Predicted_Probability=model.predict(X_img_test_normalize)

In [ ]:
def show_Predicted_Probability(X_img,Predicted_Probability,i):
    plt.figure(figsize=(2,2))
    plt.imshow(np.reshape(X_img_test[i],(32, 32,3)))
    plt.show()
    for j in range(10):
        print(label_dict[j]+' Probability:%1.9f'%(Predicted_Probability[i][j]))

In [ ]:
show_Predicted_Probability(X_img_test,Predicted_Probability,0)

# Save model

In [ ]:
# save to json file
model_json = model.to_json()
with open("SaveModel/bodyCnnModelnew.json", "w") as json_file:
    json_file.write(model_json)

# save to yaml file
model_yaml = model.to_yaml()
with open("SaveModel/bodyCnnModelnew.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

In [ ]:
# save weight 
model.save_weights("SaveModel/bodyCnnModelnew.h5")
print("Saved model to disk")